Importamos librerias y definimos la ruta de la base de datos

In [2]:
import os
import pandas as pd
import face_recognition
from PIL import Image
import matplotlib.pyplot as plt
import random
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense

# Definir la ruta de la carpeta que contiene las imágenes
ruta_carpeta = 'C:\\Users\\Waldosir\\Documents\\2doCodigo\\TopicoIA\\Marzo\\BD\\corpus_images'



Recorremos toda la carpeta y hacemos una matriz con todos los datos de todas las imagenes

In [3]:
# Crear una lista para almacenar los datos de las imágenes
datos_imagen_recortada = []
# Recorrer la carpeta para buscar archivos jpg
for carpeta_nombre in os.listdir(ruta_carpeta):
    carpeta_ruta = os.path.join(ruta_carpeta, carpeta_nombre)
    if os.path.isdir(carpeta_ruta):
        for archivo_nombre in os.listdir(carpeta_ruta):
            try:
                archivo_ruta = os.path.join(carpeta_ruta, archivo_nombre)
                
                # Cargar la imagen utilizando PIL (Pillow)
                imagen = face_recognition.load_image_file(archivo_ruta)
                
                # Redimensionar la imagen a 150x150
                imagen_redimensionada = np.array(Image.fromarray(imagen).resize((150, 150)))
                # Obtener los landmarks faciales
                landmarks = face_recognition.face_landmarks(imagen_redimensionada)
                
                caracteristicas = []
                for clave,valor in landmarks[0].items():
                    caracteristicas.append(valor)

                # Detectar los rostros en la imagen
                rostros = face_recognition.face_locations(imagen_redimensionada)

                #Recorta la cara
                cara = rostros[0]
                top, right, bottom, left = cara
                cara_recortada = imagen_redimensionada[top:bottom, left:right]
                
                # Agregar la información de la imagen al DataFrame
                datos_imagen_recortada.append([imagen_redimensionada,cara_recortada, caracteristicas,carpeta_nombre])
            except Exception as e:
                print("Error en archivo:"+str(archivo_nombre))

Error en archivo:208.jpeg
Error en archivo:225.jpeg
Error en archivo:226.jpeg
Error en archivo:228.jpeg
Error en archivo:230.jpeg
Error en archivo:234.jpeg
Error en archivo:239.jpeg
Error en archivo:240.jpeg
Error en archivo:242.jpeg
Error en archivo:243.jpeg
Error en archivo:250.jpeg
Error en archivo:251.jpeg
Error en archivo:255.jpeg
Error en archivo:256.jpeg
Error en archivo:identificador_1558_2016-11-28_14-55-22.jpeg
Error en archivo:identificador_5867_2016-12-05_15-02-28.jpeg
Error en archivo:identificador_993_2016-11-09_14-05-22.jpeg
Error en archivo:identificador_5772_2016-12-05_14-31-53.jpeg
Error en archivo:identificador_994_2016-11-09_14-05-26.jpeg


In [4]:
columnas = ['Imagen', 'Imagen con zoom', 'Hitos faciales', 'Etiquetas']
df = pd.DataFrame(datos_imagen_recortada, columns=columnas)
df.head()

,Imagen,Imagen con zoom,Hitos faciales,Etiquetas
0,"[[[165, 166, 161], [165, 166, 161], [165, 166,...","[[[160, 162, 161], [160, 162, 161], [160, 162,...","[[(62, 83), (62, 88), (62, 93), (63, 97), (64,...",bored
1,"[[[164, 166, 161], [165, 167, 162], [164, 166,...","[[[160, 162, 160], [161, 162, 161], [160, 161,...","[[(62, 83), (62, 87), (62, 92), (63, 96), (64,...",bored
2,"[[[165, 165, 163], [165, 165, 163], [164, 165,...","[[[159, 161, 158], [160, 162, 159], [161, 162,...","[[(63, 82), (63, 87), (63, 91), (64, 96), (65,...",bored
3,"[[[165, 166, 162], [165, 165, 164], [165, 165,...","[[[159, 161, 158], [161, 163, 160], [155, 157,...","[[(63, 83), (62, 87), (63, 92), (63, 97), (65,...",bored
4,"[[[165, 166, 161], [166, 167, 162], [165, 167,...","[[[160, 162, 160], [161, 163, 161], [152, 154,...","[[(63, 83), (63, 87), (63, 92), (63, 96), (64,...",bored


Conversion a JSON. Es muy pesado, por lo que pasamos de ello.

In [9]:
#df_imagenes_recortadas.to_json('test.csv')

In [10]:
#df_jason = pd.read_json('test.csv')
#df_jason.head()

In [28]:
df = pd.read_pickle('emotions_dataset.pkl')

In [29]:
df.head()


,Imagen,Imagen con zoom,Hitos faciales,Etiquetas
0,"[[[165, 166, 161], [165, 166, 161], [165, 166,...","[[[160, 162, 161], [160, 162, 161], [160, 162,...","[[(62, 83), (62, 88), (62, 93), (63, 97), (64,...",bored
1,"[[[164, 166, 161], [165, 167, 162], [164, 166,...","[[[160, 162, 160], [161, 162, 161], [160, 161,...","[[(62, 83), (62, 87), (62, 92), (63, 96), (64,...",bored
2,"[[[165, 165, 163], [165, 165, 163], [164, 165,...","[[[159, 161, 158], [160, 162, 159], [161, 162,...","[[(63, 82), (63, 87), (63, 91), (64, 96), (65,...",bored
3,"[[[165, 166, 162], [165, 165, 164], [165, 165,...","[[[159, 161, 158], [161, 163, 160], [155, 157,...","[[(63, 83), (62, 87), (63, 92), (63, 97), (65,...",bored
4,"[[[165, 166, 161], [166, 167, 162], [165, 167,...","[[[160, 162, 160], [161, 163, 161], [152, 154,...","[[(63, 83), (63, 87), (63, 92), (63, 96), (64,...",bored


In [25]:
# Función para convertir una lista de tuplas en una lista de listas
def convertir_a_lista_de_listas(lista_tuplas):
    return [[coord[0], coord[1]] for coord in lista_tuplas]

df_nuevo['Hitos faciales'] = df['Hitos faciales'].apply(convertir_a_lista_de_listas)

Sacar muestras de una fila:

In [ ]:
# Crear una nueva columna para almacenar solo los registros que cumplan con la forma deseada
df['Hitos faciales válidos'] = df['Hitos faciales'].apply(lambda x: x if np.array(x).shape == (1, 72, 2) else None)

# Eliminar los registros que no cumplen con la forma deseada
df = df.dropna(subset=['Hitos faciales válidos'])

# Convertir la columna 'Hitos faciales válidos' en un array NumPy
puntos = np.array(df['Hitos faciales válidos'].tolist())

# Verificar la forma de X
print(puntos.shape)